# 2. 온라인 선호지수 개발

## 2.2 협업 필터링을 통한 선호도 점수 행렬 도출

In [7]:
import numpy as np
import pandas as pd
import sklearn
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [2]:
#데이터가져오기(기본 데이터)
Product = pd.read_csv("C:/Users/lsh93/Desktop/data/Product.csv")
Search1 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search1.csv")
Search2 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search2.csv")
Custom = pd.read_csv("C:/Users/lsh93/Desktop/data/Custom.csv")
Session = pd.read_csv("C:/Users/lsh93/Desktop/data/Session.csv")
Master = pd.read_csv("C:/Users/lsh93/Desktop/data/Master.csv") 

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#데이터가져오기(created from 'Developing Online Preference 1')
Client_Behavior = pd.read_csv("C:/Users/lsh93/Desktop/data output/Client_Behavior.csv")

In [5]:
#첫번째 column 모두 삭제
Client_Behavior = Client_Behavior.drop(Client_Behavior.columns[[0]], axis=1)

print(Client_Behavior.columns.values[0])

CLNT_ID


### 2) 검색횟수를 이용한 온라인 선호지수 생성

In [48]:
#37만명의 clients에 대해 계산한 검색 횟수
Search_CNT = pd.read_csv("C:/Users/lsh93/Desktop/data output/Search_CNT.csv")
Search_CNT = Search_CNT.rename(columns = {'Unnamed: 0':'CLNT_ID'})
Search_CNT.head()

,CLNT_ID,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
0,4140076,24.0,53.0,15.0,5.0,16.0,14.0,7.0,2.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4148106,3.0,2.0,0.0,0.0,0.0,11.0,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5298612,2.0,5.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5553084,4.0,40.0,30.0,0.0,6.0,7.0,27.0,21.0,0.0,...,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5639922,16.0,54.0,9.0,0.0,0.0,10.0,41.0,152.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0


In [11]:
#Customer Segmentation에서 샘플링해 온 client 목록 가져오기
op_clients = pd.read_csv("C:/Users/lsh93/Desktop/data output/op_clients.csv") 
len(op_clients)

8542

In [49]:
#Purchase_Count에서 뽑아오기(두 dataframe에서  client id 교집합)
Search_CNT = pd.merge(Search_CNT, op_clients, how = 'inner', on ='CLNT_ID')
len(Search_CNT['CLNT_ID'].unique())

6450

In [50]:
#Index 추출
row_names = Search_CNT['CLNT_ID'].as_matrix()
row_names.shape

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


(6450,)

In [59]:
#Columns 추출 
col_names = Search_CNT.columns.values
col_names = col_names[1:len(col_names)-1]
col_names.shape

(37,)

In [62]:
PC = Search_CNT.as_matrix()
PC = PC[:,1:38]#1번부터 37번째 columh만,, 
PC.shape

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(6450, 37)

In [63]:
################
Search_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
Search_Count.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
4140076,24.0,53.0,15.0,5.0,16.0,14.0,7.0,2.0,7.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4148106,3.0,2.0,0.0,0.0,0.0,11.0,30.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5298612,2.0,5.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5553084,4.0,40.0,30.0,0.0,6.0,7.0,27.0,21.0,0.0,16.0,...,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5639922,16.0,54.0,9.0,0.0,0.0,10.0,41.0,152.0,0.0,60.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0


In [64]:
print(Search_Count.shape)

(6450, 37)


In [65]:
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))

Sparsity: {:4.2f}% 10.869893148962916


In [66]:
from sklearn.model_selection import train_test_split

In [67]:
#test set과 training set으로 나누기
#Array 의 형태로 나누어주는 듯 함
PC_train, PC_test = train_test_split(Search_Count, test_size = 0.33, random_state = 50)

In [68]:
print(PC_train)
print(PC_train.shape)
print(PC_test)
print(PC_test.shape)

         스포츠패션  남성의류  유아동의류  청소/세탁/욕실용품    완구  패션잡화  시즌스포츠   여성의류  인테리어/조명  \
5174505    0.0   0.0    0.0         0.0   0.0   0.0    0.0    0.0      0.0   
502099     0.0   0.0    0.0         0.0   0.0   3.0    0.0    0.0      0.0   
6139346    0.0   0.0    0.0         0.0   0.0  13.0    0.0    0.0      0.0   
853755     0.0   0.0    0.0         0.0   0.0   0.0   18.0    0.0      0.0   
1304095    1.0   0.0    0.0         0.0   0.0   0.0    0.0    0.0      0.0   
552694    20.0  30.0    0.0         0.0   0.0   0.0   18.0    0.0      0.0   
6116243    0.0   0.0    2.0         0.0   0.0   0.0    0.0    0.0      0.0   
6171962    7.0   5.0   12.0         0.0   0.0   5.0    3.0    6.0      0.0   
5294610    0.0   0.0    0.0         0.0   8.0   0.0    8.0    4.0      0.0   
4965129    0.0   0.0    0.0         0.0   0.0   0.0    0.0    0.0      0.0   
3367059    9.0   0.0    0.0         0.0   0.0   1.0    0.0    0.0      0.0   
2850244    0.0   0.0    0.0         0.0   0.0   0.0    0.0    3.

In [69]:
#DataFrame 형태로 바꾸기
PC_train = pd.DataFrame(PC_train)
PC_train.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
5174505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502099,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6139346,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
853755,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1304095,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
#DataFrame 형태로 바꾸기
PC_test = pd.DataFrame(PC_test)
PC_train.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
5174505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502099,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6139346,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
853755,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1304095,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cosine Distance 를 이용한 선호점수 계산

In [ ]:
#PC_train과 PC_test를 이용 

In [71]:
#Array 형태로 변환 
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()
print(train_mat.shape)
print(test_mat.shape)

(4321, 37)
(2129, 37)


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


#### Cosine Similarity 계산

In [72]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [73]:
#MSE 계산 함수 작성
from sklearn.metrics import mean_squared_error
def get_mse(pred,actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return mean_squared_error(pred, actual)

In [74]:
dist_out = 1 - pairwise_distances(train_mat, metric="cosine")
dist_out

array([[1.        , 0.8       , 0.57735027, ..., 0.        , 0.        ,
        0.        ],
       [0.8       , 1.        , 0.80829038, ..., 0.6       , 0.04310218,
        0.        ],
       [0.57735027, 0.80829038, 1.        , ..., 0.57735027, 0.04147509,
        0.        ],
       ...,
       [0.        , 0.6       , 0.57735027, ..., 1.        , 0.07183697,
        0.        ],
       [0.        , 0.04310218, 0.04147509, ..., 0.07183697, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [75]:
dist_out.shape

(4321, 4321)

#### w/o k means clustering

In [76]:
#Client의 알려지지 않은 평가 예측 
#거리행렬과 평가행렬의 내적을 구하고 평가수의 데이터를 정규화해 계산
user_pred = dist_out.dot(train_mat)/np.array([np.abs(dist_out).sum(axis = 1)]).T

In [77]:
#모델의 예측오차율
get_mse(user_pred, train_mat)

4878.872465841524

In [78]:
#테스트 데이터를 사용해 정확도를 확인해보기
get_mse(user_pred, test_mat)

1056.166141146887

#### w/ k means clustering

In [79]:
from sklearn.neighbors import NearestNeighbors

In [80]:
k = 5
neigh = NearestNeighbors(k, 'cosine')

In [81]:
neigh.fit(train_mat)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2,
         radius='cosine')

In [82]:
#각 사용자별로 가장 유사한 5명의 사용자 찾고 유사도 계산
top_k_distances, top_k_users = neigh.kneighbors(train_mat, return_distance = True)

In [83]:
print(top_k_distances.shape)
print(top_k_users.shape)

(4321, 5)
(4321, 5)


In [84]:
top_k_users[4]

array([408, 542,   4, 895, 898], dtype=int64)

In [85]:
#훈련 데이터의 알려지지 않은 구매 예측
user_pred_k = np.zeros(train_mat.shape)

In [87]:
for i in range(PC_train.shape[0]):
    user_pred_k[i,:] = top_k_distances[i].T.dot(train_mat[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis = 0)]).T

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [88]:
user_pred_k.shape

(4321, 37)

In [89]:
user_pred_k

array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.39473676,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [11.89943085, 25.14447862, 19.09010408, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [91]:
##### nan값이 있어서 mse를 계산할 수 없음
online_pref = pd.DataFrame(user_pred_k)
online_pref.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.0,0.0,0.0,0.0,3.50000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.394737,0.0,0.0,0.0,0.0,8.59772,0.158652,0.715796,0.0,0.394737,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.00000,19.204029,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
#nan가 들어간 element의 row 인덱스 반환
na_element_index = np.where(np.isnan(online_pref))
na_element_index
#row인덱스가 중복해서 나옴

(array([   0,    0,    0, ..., 4318, 4318, 4318], dtype=int64),
 array([ 0,  1,  2, ..., 34, 35, 36], dtype=int64))

In [94]:
#nan가 들어간 row의 인덱스 반환
na_row_index = np.unique(na_element_index[0])
na_row_index.shape

(840,)

In [96]:
#nan가 들어간 row를 삭제한다 -> 그에 해당하는 client 를 삭제한다
#제거할 client id의 목록
removed_CLNT_ID = PC_train.index.values[na_row_index]
removed_CLNT_ID.shape

(840,)

In [105]:
#제거하지 않고 남을 client id의 목록
existed_CLNT_ID = np.delete(PC_train.index.values,na_row_index)
existed_CLNT_ID.shape

(3481,)

#### nan 포함 row 제거

In [98]:
online_pref2 = online_pref.dropna()
online_pref2.shape

(3481, 37)

In [103]:
user_pred_k2 = online_pref2.as_matrix()
user_pred_k2.shape

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


(3481, 37)

In [100]:
user_pred_k2

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.39473676,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.21170228,  0.        ,  0.21170228, ...,  0.        ,
         0.        ,  0.        ],
       [11.89943085, 25.14447862, 19.09010408, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [107]:
train_mat2 = train_mat
exist_idx = np.delete([range(online_pref.shape[0])],na_row_index)
train_mat2 = train_mat2[exist_idx,:]
train_mat2.shape

(3481, 37)

In [108]:
#모델의 개선 여부 확인 -> 개선됨! 
get_mse(user_pred_k2, train_mat2)

3160.527389667947

In [110]:
get_mse(user_pred_k2, test_mat)

1561.9966024642556

- 검색횟수 + UBCF 기반 선호점수 matrix 생성 (index, column 모두 달아서)

In [111]:
col_names = Search_Count.columns.values
row_names = existed_CLNT_ID
print(col_names.shape)
print(row_names.shape)
print(user_pred_k2.shape)

(37,)
(3481,)
(3481, 37)


In [112]:
Online_Preference_Search = pd.DataFrame(user_pred_k2, columns = col_names, index = row_names)
Online_Preference_Search.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
502099,0.000000,0.000000,0.000000,0.0,0.0,3.500000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6139346,0.394737,0.000000,0.000000,0.0,0.0,8.597720,0.158652,0.715796,0.0,0.394737,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
853755,0.000000,0.000000,0.000000,0.0,0.0,0.000000,19.204029,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
552694,20.541773,29.605745,0.181459,0.0,0.0,3.649331,7.355069,1.845128,0.0,5.044275,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6171962,7.001324,4.510842,12.264304,0.0,0.0,1.986867,0.000000,2.998014,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
Online_Preference_Search.shape

(3481, 37)

In [114]:
#데이터 내보내기
Online_Preference_Search.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Online_Preference_Search.csv", encoding="UTF-8")